# Secondary structure

Protein [secondary structure](https://en.wikipedia.org/wiki/Protein_secondary_structure) is the three dimensional form of _local_ segments of proteins.

[DSSP](https://swift.cmbi.umcn.nl/gv/dssp/index.html) determines properties of the secondary structure given the three dimensional coordinates of a protein. It does not _predict_ secondary structure, just _describes_ it.

In [1]:
%matplotlib agg
import io
import os
import re
import time
import json
import hashlib
import tarfile
import requests
import tempfile
import warnings
import functools
import contextlib
import subprocess
from pathlib import Path

import bs4
import docker
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation
import natsort as ns
import tqdm.notebook as tqdm

import Bio.PDB
import Bio.SeqIO
import Bio.Align.AlignInfo
import Bio.AlignIO

from loguru import logger
from joblib import Parallel, delayed
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Markdown, HTML, Video

from graphqa.data.aminoacids import *

## [States](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6090794/)
Protein secondary structures are traditionally characterized as 3 general states: helix (H), strand (E), and coil (C).

The DSSP program uses a finer characterization of the secondary structures by extending the 3 states into 8 states:
- Helix:
  - `G` = 3-turn helix (310 helix). Min length 3 residues.
  - `H` = 4-turn helix (α helix). Minimum length 4 residues.
  - `I` = 5-turn helix (π helix). Minimum length 5 residues.
- Strand:
  - `T` = hydrogen bonded turn (3, 4 or 5 turn)
  - `E` = extended strand in parallel and/or anti-parallel β-sheet conformation. Min length 2 residues.
  - `B` = residue in isolated β-bridge (single pair β-sheet hydrogen bond formation)
  - `S` = bend (the only non-hydrogen-bond based assignment)
- Coil:
  - `-` = coil (residues which are not in any of the above conformations).

## [Dihedral angles](https://en.wikipedia.org/wiki/Dihedral_angle)
<img src="https://upload.wikimedia.org/wikipedia/commons/9/97/Protein_backbone_PhiPsiOmega_drawing.svg" style="background-color:white;width:10%;"/>

Get [DSSP](https://swift.cmbi.umcn.nl/gv/dssp/) and build a docker image
```bash
git clone https://github.com/cmbi/dssp ~/dssp
pushd ~/dssp
git checkout 697deab74011bfbd55891e9b8d5d47b8e4ef0e38
docker build -t dssp .
popd
```

Important:
- `mkdssp` only reads from file, not from stdin
- `mkdssp` skips lines shorter than 80 chars, some pdb files contains shorter lines but rewriting them with BioPython fixes the problem

Start a container with the current folder with the pdb files mounted as `/data`:

In [2]:
%%bash
(docker top dssp && docker stop dssp) 2>&1 > /dev/null
docker run --rm --tty --detach \
  --name dssp \
  --mount "type=bind,source=$(realpath ../data),target=/data" \
  'dssp'
docker ps --filter "name=dssp"

bfb3c34871f231e563bfdb2f5fe5d017b3d7ce1eba36e0e36737d3bd327d84a4
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                  PORTS               NAMES
bfb3c34871f2        dssp                "/bin/bash"         1 second ago        Up Less than a second                       dssp


In [3]:
docker_client = docker.from_env()
dssp_container = docker_client.containers.get("dssp")
df_decoys = pd.read_csv("decoys.csv")

`mkdssp` can be run inside the container, the output is buch of text that BioPython can parse for us (but only from a file on disk):

In [4]:
parser = Bio.PDB.PDBParser(QUIET=True)
structure = parser.get_structure(
    "T0759/3D-Jigsaw-V5_1_TS1", "CASP11/decoys/T0759/3D-Jigsaw-V5_1_TS1.pdb"
)

exit_code, (stdout, stderr) = dssp_container.exec_run(
    "/app/mkdssp /data/CASP11/decoys/T0759/3D-Jigsaw-V5_1_TS1.pdb", demux=True
)

# Raw output
print(
    *stdout.decode().splitlines()[:2],
    "...",
    *stdout.decode().splitlines()[27:35],
    "...",
    sep="\n",
)

# Parsed output
with tempfile.NamedTemporaryFile() as f:
    f.write(stdout)
    f.flush
    dssp = Bio.PDB.DSSP(structure[0], in_file=f.name, file_type="DSSP")

pd.DataFrame.from_dict(
    dssp.property_dict,
    orient="index",
    columns=(
        "dssp index",
        "amino acid",
        "secondary structure",
        "relative ASA",
        "phi",
        "psi",
        "NH_O_1_relidx",
        "NH_O_1_energy",
        "O_NH_1_relidx",
        "O_NH_1_energy",
        "NH_O_2_relidx",
        "NH_O_2_energy",
        "O_NH_2_relidx",
        "O_NH_2_energy",
    ),
).rename_axis(index="(chain_id, res_id)")

==== Secondary Structure Definition by the program DSSP, CMBI version 3.1.4                          ==== DATE=2020-09-21      .
REFERENCE W. KABSCH AND C.SANDER, BIOPOLYMERS 22 (1983) 2577-2637                                                              .
...
  #  RESIDUE AA STRUCTURE BP1 BP2  ACC     N-H-->O    O-->H-N    N-H-->O    O-->H-N    TCO  KAPPA ALPHA  PHI   PSI    X-CA   Y-CA   Z-CA            CHAIN AUTHCHAIN     NUMBER     RESNUM        BP1        BP2    N-H-->O    O-->H-N    N-H-->O    O-->H-N
    1   10   H              0   0  188      0, 0.0    40,-0.1     0, 0.0    34,-0.1   0.000 360.0 360.0 360.0 144.5    0.0    0.0    1.4                                     1         10          0          0          0         40          0         34
    2   11   M        +     0   0  113      1,-0.3     2,-0.2    41,-0.1    44,-0.1   0.737 360.0   9.1-116.6 -41.5   -0.8    3.2    3.3                                     2         11          0          0          1          2     

,dssp index,amino acid,secondary structure,relative ASA,phi,psi,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,NH_O_2_relidx,NH_O_2_energy,O_NH_2_relidx,O_NH_2_energy
"(chain_id, res_id)",,,,,,,,,,,,,,
"( , ( , 10, ))",1,H,-,1.000000,360.0,144.5,0,0.0,40,-0.1,0,0.0,34,-0.1
"( , ( , 11, ))",2,M,-,0.601064,-116.6,-41.5,1,-0.3,2,-0.2,41,-0.1,44,-0.1
"( , ( , 12, ))",3,V,-,0.225352,-148.5,169.8,11,-0.1,2,-0.5,12,-0.1,-1,-0.3
"( , ( , 13, ))",4,V,E,0.105634,-121.2,128.2,9,-2.2,9,-3.0,-2,-0.2,2,-0.6
"( , ( , 14, ))",5,I,E,0.698225,-97.9,126.7,-2,-0.5,7,-0.2,7,-0.2,9,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"( , ( , 105, ))",82,V,G,1.000000,-56.0,-31.8,1,-0.3,-1,-0.3,2,-0.1,-2,-0.1
"( , ( , 106, ))",83,S,G,0.453846,-95.0,9.5,-3,-1.1,-1,-0.3,-4,-0.1,-2,-0.2
"( , ( , 107, ))",84,G,T,0.797619,84.8,12.7,-3,-1.7,-3,-0.2,-4,-0.2,-2,-0.1


Two examples of problematic pdb files

In [5]:
for decoy_path in [
    "CASP9/decoys/T0515/FFAS03ss_TS4.pdb",
    "CASP9/decoys/T0515/panther_TS1.pdb",
]:
    print(decoy_path)
    print("  Original:")
    with open(decoy_path) as f:
        print(*(f"    |{l[:-1]}|" for l in f.readlines()[4:8]), sep="\n")

    structure = parser.get_structure("tmp", decoy_path)
    writer = Bio.PDB.PDBIO()
    writer.set_structure(structure)
    writer.save("/tmp/tmp.pdb", preserve_atom_numbering=True)

    print("  Rewritten:")
    with open("/tmp/tmp.pdb") as f:
        print(*(f"    |{l[:-1]}|" for l in f.readlines()[:4]), sep="\n")
    print()

CASP9/decoys/T0515/FFAS03ss_TS4.pdb
  Original:
    |ATOM      1  N   ILE     2     -37.073  18.061  41.319 1.00  0.00            N|
    |ATOM      2  CA  ILE     2     -36.036  17.502  40.423 1.00  0.00            C|
    |ATOM      3  CB  ILE     2     -35.086  18.573  39.977 1.00  0.00            C|
    |ATOM      4  CG2 ILE     2     -33.922  17.902  39.228 1.00  0.00            C|
  Rewritten:
    |ATOM      1  N   ILE     2     -37.073  18.061  41.319  1.00  0.00           N  |
    |ATOM      2  CA  ILE     2     -36.036  17.502  40.423  1.00  0.00           C  |
    |ATOM      3  CB  ILE     2     -35.086  18.573  39.977  1.00  0.00           C  |
    |ATOM      4  CG2 ILE     2     -33.922  17.902  39.228  1.00  0.00           C  |

CASP9/decoys/T0515/panther_TS1.pdb
  Original:
    |ATOM      1  N   MET     1      20.684  46.337  44.942  1.00  1.89|
    |ATOM      2  CA  MET     1      21.549  45.744  45.872  1.00  0.99|
    |ATOM      3  C   MET     1      22.654  46.773  46.2

In [ ]:
@logger.catch(reraise=False)
def run_dssp_in_docker(decoy_path: str, output_path: str):
    docker_client = docker.from_env()
    dssp_container = docker_client.containers.get("dssp")
    
    exit_code, (stdout, stderr) = dssp_container.exec_run(
        cmd=["/app/mkdssp", "/data/" + decoy_path], demux=True
    )

    if exit_code != 0:
        # Try a reformatted version of the decoy
        parser = Bio.PDB.PDBParser(QUIET=True)
        structure = parser.get_structure("tmp", decoy_path)
        
        new_decoy_path = f"{decoy_path}_tmp.pdb"
        writer = Bio.PDB.PDBIO()
        writer.set_structure(structure)
        writer.save(new_decoy_path, preserve_atom_numbering=True)
        exit_code, (stdout, stderr) = dssp_container.exec_run(
            cmd=["/app/mkdssp", "/data/" + new_decoy_path], demux=True
        )
        Path(new_decoy_path).unlink()

        if exit_code != 0:
            logger.error(
                "DSSP error {}: {}",
                decoy_path,
                stderr.decode().strip() if stderr is not None else "no stderr",
            )
            return

    with open(output_path, "wb") as f:
        f.write(stdout)

with warnings.catch_warnings():
    # Ignore PDB warnings about missing atom elements
    warnings.simplefilter("ignore", Bio.PDB.PDBExceptions.PDBConstructionWarning)

    with Parallel(n_jobs=30, verbose=1, prefer="threads") as pool:
        missing_decoys = [
            dict(
                decoy_path=f"CASP{decoy.casp_ed}/decoys/{decoy.target_id}/{decoy.decoy_id}.pdb",
                output_path=f"CASP{decoy.casp_ed}/decoys/{decoy.target_id}/{decoy.decoy_id}.dssp",
            )
            for decoy in df_decoys.itertuples()
            if not Path(
                f"CASP{decoy.casp_ed}/decoys/{decoy.target_id}/{decoy.decoy_id}.dssp"
            ).is_file()
        ]
        logger.info(f"Running on {len(missing_decoys)} .pdb files")
        pool(delayed(run_dssp_in_docker)(**decoy_dict) for decoy_dict in missing_decoys)

Check how many `.pdb` and `.dssp` files we have to see where DSSP failed

In [7]:
pdb = set(p.with_suffix("") for p in Path().glob("CASP*/decoys/*/*.pdb"))
dssp = set(p.with_suffix("") for p in Path().glob("CASP*/decoys/*/*.dssp"))
failed = pdb - dssp
if failed:
    logger.warning(
        f"DSSP failed on {len(failed)}/{len(pdb)} decoys ({len(failed)/len(pdb):.2%})"
    )

2020-09-21 22:28:21.851 | WARNING  | __main__:<module>:5 - DSSP failed on 4294/99763 decoys (4.30%)


Stop the container

In [8]:
dssp_container.stop()